
# Лабораторная 1. Аудит набора данных и экспресс-EDA для многомерных выборок. Детектирование выбросов и пропусков, сравнение критериев. Проектирование конвейера препроцессинга и документация артефактов.

**Курс:** Прикладная статистика и анализ данных   
**Раздел 1:** Современные методы описательной статистики и разведочного анализа  
**Тема:** Разведочный анализ многомерных данных, диагностика распределений и аномалий, очистка и препроцессинг в едином пайплайне.

---

## Цели ЛР

1. Научиться проводить корректный EDA для многомерных таблиц: устойчивые сводки, матрицы попарных связей, проекции, ранняя диагностика проблем качества данных.
2. Освоить диагностику формы распределений (асимметрия/тяжёлые хвосты) с использованием ECDF/QQ-плотов и робастных стандартных баллов; научиться выявлять аномалии как в одномерном, так и в многомерном варианте (через расстояние Махаланобиса и робастную ковариацию).
3. Построить воспроизводимый конвейер препроцессинга на базе Pipeline/ColumnTransformer, исключающий утечки: импутация, масштабирование, кодирование категорий, трансформации распределений; продемонстрировать корректную валидацию.
4. Включить минимальный машинно-проверяемый контроль качества входных данных (data validation) на основе декларативных ожиданий.

Ожидаемые результаты: умение (1) формулировать первичные гипотезы по структуре данных, (2) аргументированно выбирать устойчивые сводки и визуализации, (3) объяснять эффект трансформаций на форму распределений и расстояния, (4) проектировать и документировать пайплайн препроцессинга, (5) готовить воспроизводимый ноутбук с отчётом и иллюстрациями.

---

## 1. Датасет, мотивация выбора и подготовка окружения

Для обобщающей работы используется открытый набор Hotel bookings (бронирования отелей). Он богат числовыми и категориальными признаками (включая даты, длительности, цену/ADR), содержит пропуски и нетривиальные распределения (правые хвосты по цене, разную сезонность), что делает его существенно более реалистичным, чем «игрушечные» учебные наборы. Широко используется производная версия из сообщества TidyTuesday (CSV) — удобно загружается напрямую в pandas.

> Замечание об источнике: для воспроизводимости берём «плоский» CSV TidyTuesday: https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv (содержит столбцы бронирований для двух типов отелей). Краткое описание структуры и происхождения см. в репозитории TidyTuesday и статье Data in Brief.

---

In [ ]:

# Подготовка окружения и загрузка:

# Импорт базовых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Статистика/моделирование
from statsmodels.distributions.empirical_distribution import ECDF  # ECDF-кривые
from statsmodels.graphics.gofplots import qqplot                     # QQ-плоты
from sklearn.covariance import MinCovDet                             # робастная ковариация (MCD)
from sklearn.preprocessing import RobustScaler, QuantileTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

# Графические параметры
plt.rcParams["figure.figsize"] = (8, 5)
sns.set(style="whitegrid")

# Загрузка набора данных (CSV TidyTuesday)
URL = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv"
df = pd.read_csv(URL)  # pandas.read_csv поддерживает URL-источники
df.shape, df.head(3)



## 2. Часть 1. Разведочный анализ (EDA): устойчивые сводки и визуализация многомерности

Убедиться, что строки/категории и числа распознаны корректно; явные даты соберите из компонент arrival_date_year, arrival_date_month, arrival_date_day_of_month в единый столбец (опционально).

Для числовых колонок рассчитайте устойчивые сводки: медиана $\tilde{x}$, межквартильный размах $IQR=Q_{0.75}-Q_{0.25}$, медианное абсолютное отклонение $MAD=\mathrm{median}(|x_i-\tilde{x}|)$.

Сформируйте таблицу устойчивых сводок по ключевым числовым полям: lead_time, stays_in_weekend_nights, stays_in_week_nights, adults, children, babies, adr (average daily rate).


In [ ]:
# Преобразование месяца в номер и сборка даты заезда (для иллюстраций сезонности)
month_map = {m:i for i, m in enumerate(
    ["January","February","March","April","May","June","July","August","September","October","November","December"], start=1)}
df["arrival_month_num"] = df["arrival_date_month"].map(month_map)
df["arrival_date"] = pd.to_datetime(dict(year=df["arrival_date_year"],
                                         month=df["arrival_month_num"],
                                         day=df["arrival_date_day_of_month"]), errors="coerce")

# Устойчивые сводки по выбранным числовым признакам
num_cols = ["lead_time","stays_in_weekend_nights","stays_in_week_nights","adults","children","babies","adr"]
def robust_summary(s: pd.Series):
    s = s.dropna()
    med = np.median(s)
    q1, q3 = np.percentile(s, [25, 75])
    iqr = q3 - q1
    mad = np.median(np.abs(s - med))
    return pd.Series({"count": s.size, "median": med, "q1": q1, "q3": q3, "IQR": iqr, "MAD": mad})

robust_tbl = df[num_cols].apply(robust_summary, axis=0).T
robust_tbl


### Матрицы парных связей и тепловые карты корреляций

Постройте pairplot для поднабора числовых признаков (включая $\log(\text{adr}+1)$ для стабилизации правого хвоста).

Постройте тепловую карту корреляций (Пирсон и Спирмен) и сравните паттерны (кластеры, мультиколлинеарность).

In [ ]:
# Лёгкая стабилизация "правого хвоста" цены
df["log_adr"] = np.log1p(df["adr"].clip(lower=0))

# Pairplot для обзорной разведки
sns.pairplot(df[["lead_time","stays_in_week_nights","stays_in_weekend_nights","adults","log_adr"]]
             .dropna().sample(5000, random_state=42), diag_kind="hist", plot_kws=dict(alpha=0.3, s=10))
plt.suptitle("Матрица парных диаграмм (подвыборка 5k)", y=1.02)
plt.show()

# Тепловая карта корреляций (Пирсон, затем Спирмен)
corr_p = df[num_cols].corr(method="pearson")
corr_s = df[num_cols].corr(method="spearman")
fig, axes = plt.subplots(1,2, figsize=(14,5))
sns.heatmap(corr_p, ax=axes[0], vmin=-1, vmax=1, annot=False)
axes[0].set_title("Корреляции Пирсона")
sns.heatmap(corr_s, ax=axes[1], vmin=-1, vmax=1, annot=False)
axes[1].set_title("Корреляции Спирмена")
plt.show()


#Пояснение: seaborn.pairplot и тепловая карта — стандартные инструменты первого взгляда на структуру признаков. С практической точки зрения полезно сопоставлять Пирсона (линейная зависимость) и Спирмена (монотонная) при правых хвостах и выбросах.

## 3. Часть 2. Исследование распределений и аномалий

### ECDF и интерпретация хвостов

Эмпирическая CDF (ECDF) определяется как $ \hat{F}(x)=\frac{1}{n}\sum_{i=1}^n \mathbf{1}{X_i \le x} $. Постройте ECDF для adr раздельно по типу отеля (hotel), отметьте вертикалями перцентили $P_{50},P_{90},P_{99}$ и SLA-пороги (если применимо).

In [ ]:
def plot_ecdf_by_group(df, value, group):
    plt.figure()
    for g, dfg in df[[value, group]].dropna().groupby(group):
        ecdf = ECDF(dfg[value].values)
        xs = np.linspace(dfg[value].min(), dfg[value].quantile(0.995), 400)
        ys = ecdf(xs)
        plt.plot(xs, ys, label=str(g))
    for q in [0.5, 0.9, 0.99]:
        plt.axvline(df[value].quantile(q), ls="--", alpha=0.4)
    plt.xlabel(value); plt.ylabel("ECDF")
    plt.title(f"ECDF {value} по группам {group}")
    plt.legend()
    plt.show()

plot_ecdf_by_group(df, "adr", "hotel")


# Пояснение: ECDF позволяет читать хвостовые доли без «бининга» и сразу сравнивать группы; реализация доступна в statsmodels.

### QQ-плоты и выбор трансформаций

QQ-плот сравнивает квантили выборки с теоретическим распределением. Для adr постройте два QQ-плота: против нормального закона для $adr$ и для $\log(adr+1)$. Наблюдения о «выпрямлении» линий обосновывают применение степенных/лог-трансформаций перед моделированием.

In [ ]:
# QQ-плот для adr и log_adr
fig, axes = plt.subplots(1,2, figsize=(12,5))
qqplot(df["adr"].dropna().clip(upper=df["adr"].quantile(0.999)), line="s", ax=axes[0])
axes[0].set_title("QQ: adr vs Normal")
qqplot(df["log_adr"].dropna(), line="s", ax=axes[1])
axes[1].set_title("QQ: log(adr+1) vs Normal")
plt.show()

### Робастные z-оценки и одномерные «выбросы»

Определим модифицированную $z$-оценку $ z_i^{(MAD)} = \dfrac{x_i-\tilde{x}}{1.4826\cdot MAD} $ и пометим наблюдения с $|z_i^{(MAD)}|>3.5$ как «подозрительные» для аудита.

In [ ]:
def robust_z(x):
    x = np.asarray(x[~np.isnan(x)])
    med = np.median(x)
    mad = np.median(np.abs(x - med))
    return (x - med) / (1.4826 * (mad + 1e-12))

z = robust_z(df["adr"].values)
np.mean(np.abs(z)>3.5)

### Многомерные аномалии: расстояние Махаланобиса с робастной ковариацией (MCD)

Расстояние Махаланобиса $ d_i=\sqrt{(x_i-\hat{\mu})^\top \hat{\Sigma}^{-1} (x_i-\hat{\mu})} $ при робастных оценках $(\hat{\mu},\hat{\Sigma})$ (алгоритм Minimum Covariance Determinant) уменьшает влияние «хвостов» и «рычагов». Модель MinCovDet реализована в scikit-learn. Для числового подпространства оцените $d_i^2$ и отметьте точки с $d_i^2>\chi^2_{p,,0.995}$.

In [ ]:
from scipy.stats import chi2

X = df[["lead_time","stays_in_week_nights","stays_in_weekend_nights","adults","log_adr"]].dropna().values
mcd = MinCovDet(random_state=42).fit(X)  # FAST-MCD
d2 = mcd.mahalanobis(X)                  # квадраты расстояний
thr = chi2.ppf(0.995, df=X.shape[1])     # порог по chi2(p)
np.mean(d2 > thr), thr



> Пояснение: MinCovDet даёт робастные оценки центра/ковариации; квадраты расстояний сопоставимы с квантилями $\chi^2_p$.


## Часть 3. Очистка и препроцессинг в едином пайплайне (без утечки)

Постановка: подготовить признаки для простой задачи бинарной классификации отмены бронирования (is_canceled) с корректной обработкой пропусков и категорий; цель — не «высокий скор», а правильно устроенный конвейер.

Шаги:
1. Разделите признаки на числовые и категориальные; для числовых примените импутацию (медиана) и робастное масштабирование (RobustScaler), устойчивое к хвостам $IQR$; для сильно асимметричных величин можно использовать QuantileTransformer (к нормальному распределению).
2. Для категориальных — импутация «most_frequent» и OneHotEncoder(handle_unknown="ignore").
3. Соберите всё через ColumnTransformer и Pipeline; проверьте метрику на валидации (StratifiedKFold).

In [ ]:
# Целевая переменная и поднабор признаков
y = df["is_canceled"].astype(int)
features_num = ["lead_time","stays_in_week_nights","stays_in_weekend_nights","adults","children","babies","adr"]
features_cat = ["hotel","meal","market_segment","distribution_channel","reserved_room_type","customer_type","deposit_type","country"]

X = df[features_num + features_cat]

# Простейшая стратегия импутации
num_pipe = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="median")),
    ("scale", RobustScaler())  # масштаб по медиане и IQR
])

cat_pipe = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", min_frequency=20))
])

pre = ColumnTransformer([
    ("num", num_pipe, features_num),
    ("cat", cat_pipe, features_cat)
])

clf = Pipeline(steps=[
    ("pre", pre),
    ("est", LogisticRegression(max_iter=2000, n_jobs=None))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(clf, X, y, cv=cv, scoring="roc_auc")
scores.mean(), scores.std()


> Пояснение: RobustScaler центрирует по медиане и масштабирует по $IQR$, снижая влияние выбросов; QuantileTransformer (по желанию) может дополнительно «выпрямить» маргинальные распределения (но меняет геометрию расстояний). ColumnTransformer/Pipeline предотвращают утечки, обучая все шаги только на обучающей части.


## 5. Мини-валидация качества входных данных (data validation)

Перед обучением полезно проверять простейшие инварианты набора: уникальность идентификаторов (если есть), домен значений, отсутствие отрицательных длительностей/количеств, адекватность валют и т. п. В промышленных пайплайнах применяют декларативные ожидания (expectations), которые автоматически валидируют таблицу и формируют отчёт. Пример ниже — «микро-suite» в стиле Great Expectations (идея/терминология); его можно встроить в ноутбук/скрипт как быстрый гейт на входе.

In [ ]:
# Простейшие проверки "как ожидания": возвращают True/False и список нарушений
def expect_not_null(s: pd.Series, name: str):
    bad = s[s.isna()]
    return bad.empty, bad.index.tolist()

def expect_ge(s: pd.Series, name: str, min_value: float):
    bad = s[s < min_value]
    return bad.empty, bad.index.tolist()

checks = {
    "adr_nonneg": expect_ge(df["adr"].fillna(0), "adr", 0.0),
    "adults_nonneg": expect_ge(df["adults"].fillna(0), "adults", 0.0),
    "lead_time_nonneg": expect_ge(df["lead_time"].fillna(0), "lead_time", 0.0),
    "hotel_not_null": expect_not_null(df["hotel"], "hotel")
}

{key: ok for key, (ok, idx) in checks.items()}

> Пояснение: В полномасштабных проектах такие правила оформляют как «expectation suites» (например, Great Expectations) с HTML-отчётами и историей прогонов.

## 6. Формулы и блок-схема работы

Ключевые формулы:
- Эмпирическая CDF: $ \hat{F}(x)=\tfrac{1}{n}\sum_{i=1}^n \mathbf{1}{X_i\le x} $.
- Межквартильный размах: $ \mathrm{IQR}=Q_{0.75}-Q_{0.25} $.
- MAD: $ \mathrm{MAD}=\mathrm{median}(|x_i-\tilde{x}|) $, робастный масштаб $ \hat{\sigma}_{MAD}=1.4826\cdot \mathrm{MAD} $.
- Робастная $z$-оценка: $ z_i^{(MAD)}=\dfrac{x_i-\tilde{x}}{1.4826\cdot \mathrm{MAD}} $.
- Расстояние Махаланобиса: $ d_i=\sqrt{(x_i-\hat{\mu})^\top \hat{\Sigma}^{-1}(x_i-\hat{\mu})} $.
- Порог по $\chi^2$: $ d_i^2>\chi^2_{p,,1-\alpha} $ — «аномалия» на уровне значимости $\alpha$.


### Блок-схема (ASCII):
    
    - A[[Загрузка CSV]]
    
    - B[Быстрые проверки: типы / NA]
    
    - C[EDA: устойчивые сводки;<br/>корреляции; pairplot]
    
    - D[ECDF, QQ: диагностика формы]
    
    - E[Outliers: z(MAD), MCD / Махаланобис]
    
    - F[Пайплайн препроцессинга:<br/>импутация → масштаб/преобр. → OHE]
    
    - G[Валидация данных (expectations)]
    
    - H[Кросс-валидация модели]

    A --> B --> C
    C --> D
    C --> E
    D --> F
    E --> F
    F --> G --> H



## 7. Контрольные задания

### 1. EDA и устойчивые сводки.
Сформируйте таблицу устойчивых сводок по выбранным числовым признакам (см. § 2.1), прокомментируйте различия «медиана/MAD/IQR» vs «среднее/SD» в правохвостых столбцах (напр., adr, lead_time). Объясните, почему медиана/IQR устойчивее при наличии редких экстремумов, и как это видно на гистограммах/ECDF. Обязательно включите матрицу корреляций (Пирсон и Спирмен) и разберите не менее двух несоответствий между ними: где линейная связь слабая, а монотонная - выражена (или наоборот). Сформулируйте как минимум три первичные гипотезы о структуре данных: групповые различия (тип отеля), сезонность по месяцу приезда, потенциальные взаимодействия признаков (например, lead_time × сезон). Сохраните рисунки (pairplot, heatmap) и кратко опишите интерпретацию.

### 2. Форма распределений и трансформации.
Постройте ECDF для adr (цены/сутки) раздельно по типу отеля и отметьте $P_{50},P_{90},P_{99}$. Сравните хвосты: есть ли систематический сдвиг у одной из групп? Добавьте QQ-плоты adr и $\log(adr+1)$ против нормального закона; оцените, насколько лог-трансформация «выпрямляет» хвосты и среднюю часть. Обоснуйте, какие признаки в дальнейшем разумно трансформировать (лог/степень/квантили) и почему. В отчёте отразите формулы ECDF и аргументы про хвостовые доли (например, долю наблюдений с adr выше $P_{95}$). Сопоставьте выводы с корреляционными матрицами: как трансформации влияют на линейные коэффициенты? Приведите не менее двух иллюстраций с пояснениями.

### 3. Одномерные и многомерные аномалии.
Для adr вычислите модифицированные $z$-оценки $z^{(MAD)}$ и дайте оценку доли наблюдений с $|z|>3.5$. Проверьте несколько «аномальных» строк на предмет ошибок/редких режимов (например, высокий adr при коротком lead_time). В многомерном подпространстве оцените квадраты расстояний Махаланобиса на базе робастной ковариации (MCD) и пометьте точки с $d^2>\chi^2_{p,0.995}$. Сравните наборы «аномальных» наблюдений из одномерного и многомерного подходов: где методы согласуются, а где — выявляют разные случаи? Объясните, почему многомерная метка может отличаться (корреляции, другая геометрия). Приложите диаграмму рассеяния с цветовой пометкой «аномалий» и прокомментируйте.

### 4. Пайплайн препроцессинга и валидация.
Соберите Pipeline + ColumnTransformer для задачи is_canceled (см. § 4), добейтесь корректной кросс-валидации (StratifiedKFold) и отчётливо объясните, почему такая организация исключает утечки (все статистики fit обучаются только на тренировочных фолдах). Сравните две конфигурации: (A) RobustScaler для чисел; (B) та же схема + QuantileTransformer для сильно асимметричных чисел (например, adr, lead_time). Приведите среднее и стандартное отклонение ROC-AUC по фолдам, а также поясните отличия. Добавьте «микро-suite» проверок входных данных (см. § 5), опишите, какие нарушения он ловит на этом наборе. Сохраните финальный рисунок или таблицу с результатами.

## 8. Критерии оценивания

Воспроизводимость (10 %) — ноутбук запускается «с нуля»; ссылки на источник данных и версии библиотек указаны; установлен фиксированный random_state.

Корректный EDA (25 %) — устойчивые сводки, матрицы корреляций (Пирсон/Спирмен), интерпретации по группам/сезонам; аккуратность графиков (легенды, подписи, осмысленные лимиты).

Диагностика распределений (20 %) — ECDF/QQ-плоты с внятными выводами о хвостах и трансформациях; корректные формулы/обоснования.

Аномалии (15 %) — корректная реализация $z^{(MAD)}$ и MCD-Махаланобиса, сравнение одномерного и многомерного кейсов с предметными комментариями.

Пайплайн и валидация (20 %) — отсутствие утечек, понятная разметка числовых/категориальных, адекватные трансформеры, кросс-валидация, краткий анализ метрик; наличие мини-валидации качества входных данных.

Отчётность и стиль (10 %) — готовый отчет, академический слог, структурированность (рисунки, таблицы, формулы), равернутые и понятные подписи и интерпретации.

## 9. Приложение: дополнительные фрагменты кода

In [ ]:
# Подсветка многомерных аномалий на парных графиках

# Метки "аномалий" по порогу chi2 для визуализации
mask = d2 > thr
X_df = pd.DataFrame(X, columns=["lead_time","stays_wd","stays_we","adults","log_adr"])
X_df["outlier"] = np.where(mask, "anom", "ok")
sns.pairplot(X_df.sample(5000, random_state=42), hue="outlier", plot_kws=dict(alpha=0.4, s=10))
plt.suptitle("Многомерные аномалии по робастному Махаланобису (подвыборка)", y=1.02)
plt.show()


In [ ]:
# Вариант с QuantileTransformer для «тяжёлых» чисел

skewed = ["lead_time","adr"]
num_pipe_qt = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("qt", QuantileTransformer(output_distribution="normal", random_state=42))
])

pre_qt = ColumnTransformer([
    ("num_qt", num_pipe_qt, skewed),
    ("num_rs", Pipeline([("imp", SimpleImputer(strategy="median")), ("scale", RobustScaler())]),
     list(set(features_num) - set(skewed))),
    ("cat", cat_pipe, features_cat)
])

clf_qt = Pipeline([("pre", pre_qt), ("est", LogisticRegression(max_iter=2000))])
scores_qt = cross_val_score(clf_qt, X, y, cv=cv, scoring="roc_auc")
scores.mean(), scores.std(), scores_qt.mean(), scores_qt.std()


> Пояснение: QuantileTransformer приводит маргинальные распределения к Uniform/Normal, «сжимая» хвосты; полезно на правохвостых признаках, однако меняет метрику расстояний — интерпретируйте аккуратно.